# Loading data

In [1]:
import numpy as np
import pandas as pd

calendar = pd.read_csv("data/calendar.csv")
sales = pd.read_csv("data/sales_train_validation.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")
sell_prices = pd.read_csv("data/sell_prices.csv")

In [2]:
calendar[calendar["d"]=="d_1904"]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1903,2016-04-15,11611,Friday,7,4,2016,d_1904,NaN,NaN,NaN,NaN,0,1,1


# Loading your data
Please make sure that it complies the same format here.

In [3]:
sample = pd.read_csv("sample.csv")
sample.head()

,id,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0


In [5]:
my_sample = pd.read_csv("internal_submission.csv")
my_sample.drop(["Unnamed: 0"], axis=1, inplace=True)
my_sample.columns = sample.columns
my_sample.head()

,id,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,FOODS_1_001_CA_1_validation,3.407692,2.910621,1.896974,5.476263,13.310259,15.778389,8.387853,1.333270,0.289329,0.069948
1,FOODS_1_001_CA_2_validation,2.978067,3.438599,3.565418,4.599385,11.705769,17.651206,12.549229,2.820791,0.365000,0.099835
2,FOODS_1_001_CA_3_validation,2.947160,4.419345,3.474644,3.888860,10.181584,19.713310,18.466656,5.761748,1.233935,0.260625
3,FOODS_1_001_CA_4_validation,1.820875,4.997311,4.444072,3.277345,7.716993,20.523785,23.660580,8.874112,1.648879,0.180767
4,FOODS_1_001_TX_1_validation,0.983622,4.401130,5.990968,3.983368,5.714690,13.322697,20.603277,28.773722,6.446603,0.614928


# RMSE

In [6]:
groudtruth = sales[["id", "d_1904", "d_1905", "d_1906", "d_1907", "d_1908", "d_1909", "d_1910", "d_1911", "d_1912", "d_1913"]]

def transform(df):
    newdf = df.melt(id_vars=["id"], var_name="d", value_name="sale")
    newdf.sort_values(by=['id', "d"], inplace=True)
    newdf.reset_index(inplace=True)
    return newdf

from sklearn.metrics import mean_squared_error

def rmse(df, gt):
    df = transform(df)
    gt = transform(gt)
    return mean_squared_error(df["sale"], gt["sale"])


In [7]:
rmse(sample, groudtruth), rmse(my_sample, groudtruth)

(14.980924893407675, 1329.8677419980356)

# WRMSSE

In [8]:
# import warnings
# warnings.filterwarnings('ignore')
# import dask.dataframe as dd
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
# import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
# import lightgbm as lgb
# #import dask_xgboost as xgb
# #import dask.dataframe as dd
# from sklearn import preprocessing, metrics
# from sklearn.preprocessing import LabelEncoder
import gc
# import os
# from tqdm import tqdm
from scipy.sparse import csr_matrix


In [9]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: 
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [10]:
NUM_ITEMS = sales.shape[0]  # 30490
DAYS_PRED = sample_submission.shape[1] - 1  # 28

In [17]:
idcols = ["id", "item_id", "state_id", "store_id", "cat_id", "dept_id"]
daycols = ["d_1904", "d_1905", "d_1906", "d_1907", "d_1908", "d_1909", "d_1910", "d_1911", "d_1912", "d_1913"]
product = (sales[idcols])
product.head()

,id,item_id,state_id,store_id,cat_id,dept_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA,CA_1,HOBBIES,HOBBIES_1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA,CA_1,HOBBIES,HOBBIES_1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA,CA_1,HOBBIES,HOBBIES_1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,CA,CA_1,HOBBIES,HOBBIES_1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,CA,CA_1,HOBBIES,HOBBIES_1


In [30]:
pd.get_dummies(product.state_id, drop_first=False)
weight_mat = np.c_[
   np.ones([NUM_ITEMS,1]).astype(np.int8), # level 1
   pd.get_dummies(product.state_id, drop_first=False).values,
   pd.get_dummies(product.store_id, drop_first=False).values,
   pd.get_dummies(product.cat_id, drop_first=False).values,
   pd.get_dummies(product.dept_id, drop_first=False).values,
   pd.get_dummies(product.state_id + product.cat_id, drop_first=False).values,
   pd.get_dummies(product.state_id + product.dept_id, drop_first=False).values,
   pd.get_dummies(product.store_id + product.cat_id, drop_first=False).values,
   pd.get_dummies(product.store_id + product.dept_id, drop_first=False).values,
   pd.get_dummies(product.item_id, drop_first=False).values,
   pd.get_dummies(product.state_id + product.item_id, drop_first=False).values,
   np.identity(NUM_ITEMS).astype(np.int8) #item :level 12
].T

weight_mat = weight_mat.astype("int8")
weight_mat, weight_mat.shape

(array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 1]], dtype=int8),
 (42840, 30490))

In [31]:
weight_mat_csr = csr_matrix(weight_mat)
del weight_mat; gc.collect()


732

In [63]:
np.save("w_csr", [weight_mat_csr])
weight_mat_csr2 = np.load("w_csr.npy", allow_pickle=True)[0]

In [66]:
# weight_mat_csr.shape, weight_mat_csr2.shape

In [100]:
# def cal_weight1(product):
#     sales_train_val = sales
#     d_name = ['d_' + str(i+1) for i in range(1913)]

#     sales_train_val = weight_mat_csr * sales_train_val[d_name].values


#     df_tmp = ((sales_train_val>0) * np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))

#     start_no = np.min(np.where(df_tmp==0,9999,df_tmp),axis=1)-1

#     flag = np.dot(np.diag(1/(start_no+1)) , np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))<1

#     sales_train_val = np.where(flag, np.nan, sales_train_val)

#     # denominator of RMSSE / RMSSE
#     weight1 = np.nansum(np.diff(sales_train_val,axis=1)**2,axis=1)/(1913-start_no)
    
#     return weight1

# weight1 = cal_weight1(product)

In [101]:
# sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [102]:
# # Get the last 28 days for weight2
# cols = ["d_{}".format(i) for i in range(1886, 1886+28)]

# data = sales[["id", 'store_id', 'item_id'] + cols]

# data = data.melt(id_vars=["id", 'store_id', 'item_id'], var_name="d", value_name="sale")
# data = pd.merge(data, calendar, how = 'left', left_on = ['d'], right_on = ['d'])
# data = data[["id", 'store_id', 'item_id', "sale", "wm_yr_wk"]]
# data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
# data.head()

,id,store_id,item_id,sale,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,1,11609,8.26
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,1,11609,3.97
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,0,11609,2.97
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,0,11609,4.64
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,1,11609,2.88


In [103]:

# def cal_weight2(data):
#     # calculate the sales amount for each item/level
#     df_tmp = data
#     df_tmp['amount'] = df_tmp['sale'] * df_tmp['sell_price']
#     df_tmp =df_tmp.groupby(['id'])['amount'].apply(np.sum)
#     df_tmp = df_tmp.values
    
#     weight2 = weight_mat_csr * df_tmp 

#     weight2 = weight2/np.sum(weight2)
#     return weight2
    
# weight2 = cal_weight2(data)
# weight2.shape

(42840,)

In [109]:
np.save("w1", weight1)
np.save("w2", weight2)

### Load weight1, weight2

In [110]:
weight1 = np.load("w1.npy")
weight2 = np.load("w2.npy")

In [111]:
def wrmsse(preds, y_true):
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
    
          
    train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) / weight1) * weight2)
    
    return score

In [112]:
# rmse(sample, groudtruth)
DAYS_PRED = 10
df = transform(sample)
gt = transform(groudtruth)
wrmsse(df["sale"].to_numpy(), gt["sale"].to_numpy())

7.627457792062966